In [11]:
import pandas as pd
import numpy as np

layers = [8, 25, 3]
weights = []
bias = []
colNames = ['class', 'ra', 'dec', 'u', 'g', 'r', 'i', 'z', 'nuv_mag']
dataSet = pd.read_csv('GALEX_data-extended-feats.csv', usecols = colNames)

In [12]:
def sig(z):
    return (1)/(1 + np.exp(-z))

def sig_prime(z):
    return sig(z)*(1-sig(z))

def initialiser(layers, weights, bias):
    # initialise all weights and biases throughout the network to small random values
    nLayers = len(layers)
    for num in range(1, nLayers):
        for num2 in range(layers[num]):
            layWeights = np.random.uniform(-0.05, 0.05, layers[num - 1])
            weights.append(layWeights)
    bias.append(np.random.uniform(-0.05, 0.05, sum(layers[1:])))
    return weights, bias



initialiser(layers, weights, bias)

([array([ 0.02120881, -0.04066429, -0.04828364, -0.04750896, -0.01406252,
         -0.04402857,  0.03276342,  0.03465108]),
  array([ 0.04090764, -0.04552518,  0.00580651,  0.02843066, -0.03350593,
         -0.01206124, -0.02154532,  0.00554491]),
  array([ 0.01109053,  0.0426739 , -0.0452781 ,  0.01387197,  0.03834411,
          0.04768234, -0.02608262,  0.01575466]),
  array([ 0.02193718,  0.03732913, -0.02475438, -0.02029796,  0.00308046,
         -0.02182926,  0.00166826, -0.02616911]),
  array([-0.01551041,  0.00175619,  0.01988498, -0.005308  , -0.00454734,
         -0.04908239, -0.02991329,  0.00177247]),
  array([-0.00464619,  0.02239562,  0.03543637, -0.00053482, -0.04385546,
          0.03672666, -0.00319929, -0.0196541 ]),
  array([-0.04899323, -0.03802114, -0.0192856 ,  0.00916419, -0.0129543 ,
          0.03245915, -0.04166398,  0.03176194]),
  array([-0.01086429, -0.04190839,  0.01678812, -0.01650703,  0.04763678,
         -0.03006276, -0.04222601, -0.01438869]),
  array(

In [13]:
def feedFor(x, i, w, b):
    # l should be a two element array, input vector and output vector, ex: [8, 5] or [5, 3]
    base = sum(layers[1:i])
    ele = layers[i]
    newOut = []
    for num in range(ele):
        sigVal = sig(np.dot(w[base + num], x) + b[0][base + num])
        newOut.append(sigVal)
    return newOut

def fixWeights(outputk, outputh, i, w):
    if(dataSet.iloc[i, 0] == 0):
        exp = [1, 0, 0]
    elif(dataSet.iloc[i, 0] == 1):
        exp = [0, 1, 0]
    else:
        exp = [0, 0, 1]
    delK = []; delH = []
    for num in range(layers[-1]):
        delK.append(outputk[num]*(1 - outputk[num])*(exp[num] - outputk[num]))
    for num1 in range(layers[-2]):
        tSum = 0
        for num2 in range(layers[-1]):
            tSum += weights[num1][num2] * delK[num2] 
        delH.append(tSum)
    return delH, delK
        
def all(eta):
    for num in range(4000):
        currRow = dataSet.iloc[num, 1:]
        outputh = feedFor(currRow, 1, weights, bias)
        outputk = feedFor(outputh, 2, weights, bias)
        delH, delK = fixWeights(outputk, outputh, num, weights)
        for num1 in range(layers[1]):
            for num2 in range(layers[0]):
                weights[num1][num2] += (eta)*(delH[num1])*(currRow[num2])
            bias[0][num1] += (eta)*(delH[num1])
        for num1 in range(layers[1], layers[1] + 3):
            for num2 in range(25):
                weights[num1][num2] += (eta)*(delK[num1 - 25])*(outputh[num2])
            bias[0][num1] += (eta)*(delK[num1 - 25])
      
all(0.1)
print(*weights, *bias)


c:\users\gaurang\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in exp
  


[8744.02507235 1475.32746091 1055.92989184  770.30561787  603.09024144
  502.04497672  509.62764406  715.03907659] [-36939.12196636  -6232.51884497  -4460.77431251  -3254.44357784
  -2548.01765727  -2121.26546448  -2152.98009443  -3020.75516685] [60680.4553575  10238.25314142  7327.78156903  5346.1506579
  4185.60958127  3484.62356859  3536.6342591   4962.20900047] [40825.55031233  6888.25283653  4930.1000346   3596.8367323
  2816.03902101  2344.38874705  2379.45478297  3338.51752159] [-12640.62975414  -2132.778446    -1526.48449059  -1113.67649071
   -871.91030864   -725.92715468   -736.75827428  -1033.68225515] [-12092.33907921  -2040.2686139   -1460.27282499  -1065.36118317
   -834.1202049    -694.3440129    -704.76337137   -988.85462176] [-5695.83941756  -961.02518478  -687.83048094  -501.81106296
  -392.90488032  -327.06443627  -332.02864534  -465.7589507 ] [-38646.126492    -6520.53158861  -4666.91246013  -3404.86475281
  -2665.6642989   -2219.29375094  -2252.47750072  -3160.3429

In [14]:

np.save('weights', weights)
np.save('bias', bias)